# Master Donor Feature Schema for DonorsChoose

## Overview

This notebook implements a comprehensive, reusable donor feature engineering pipeline that:

1. **Re-uses existing feature engineering work** while consolidating near-duplicate concepts
2. **Separates features by time scope**:
   - `STATIC`: Does not depend on reference time T (or treated as fixed once per donor)
   - `AS_OF_T`: Uses cumulative data up to time T
   - `WINDOWED`: Uses specific lookback windows relative to T
   - `LABEL`: Natural target variables for various use cases

3. **Pulls from 5 main data sources**:
   - Donor Project Records (gifts/donations)
   - Email Events (12-month summary)
   - Site Events (FY25-26 activity)
   - Monthly Donation Program (all-time)
   - Share Events (all-time)
   - Plus: ZIP-level ACS demographics

## Feature Categories

The schema includes approximately 150+ features across these domains:

1. **Identity & Demographics** (ZIP-based ACS)
2. **Lifetime Giving Behavior** (tenure, amounts, patterns)
3. **Windowed Giving** (3m, 12m, 36m activity + velocity)
4. **Channel/Payment Mix** (DAF, green, gift cards, etc.)
5. **Monthly Program Dynamics** (subscription behavior)
6. **Teacher/School/Content Preferences** (loyalty vs diversification)
7. **Seasonality** (back-to-school, year-end, etc.)
8. **Email Engagement** (opens, clicks, velocity)
9. **Site Behavior** (sessions, pages, devices)
10. **Share Activity** (social sharing patterns)
11. **Project Outcomes** (fully-funded rates, matching)
12. **Labels** (optional, for various prediction tasks)

## Key Design Principles

- **No duplicate concepts**: Single distance metric, single velocity calculation pattern
- **Consistent windowing**: 3-month (short), 12-month (mid), 36-month (long)
- **Defensive coding**: Small epsilon values prevent division by zero
- **Separation of concerns**: Feature building ≠ imputation/encoding
- **Time-relative**: All features parameterized by reference time T


## Setup and Imports

In [1]:
import numpy as np
import pandas as pd
from pyproj import Geod
from typing import Optional, Tuple
import warnings
# warnings.filterwarnings('ignore')

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

## 1. Helper Functions

These utility functions support the main feature engineering pipeline.

In [2]:
def haversine_miles(lat1, lon1, lat2, lon2):
    """
    Compute great-circle distance between two points in miles using Haversine formula.
    
    All arguments can be scalars or pandas Series (for vectorized computation).
    
    Parameters
    ----------
    lat1, lon1 : float or pd.Series
        Latitude and longitude of first point(s) in decimal degrees
    lat2, lon2 : float or pd.Series
        Latitude and longitude of second point(s) in decimal degrees
        
    Returns
    -------
    distance : float or pd.Series
        Great-circle distance in miles
    """
    # Earth's radius in miles
    R = 3958.8
    
    # Convert all coordinates from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

def entropy_vectorized(df, group_col, cat_col):
    """
    Calculate Shannon entropy in a fully vectorized way.
        
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the data
    group_col : str
        Column to group by (e.g., 'donor_id')
    cat_col : str
        Categorical column to calculate entropy for (e.g., 'teacher_id')
    
    Returns
    -------
    entropy : pd.Series
        Entropy value for each group, indexed by group_col
    """
    if df.empty:
        return pd.Series(dtype=float, name='entropy')
    
    # Count occurrences of each category within each group
    counts = df.groupby([group_col, cat_col]).size()
    
    # Calculate probabilities (normalize within each group)
    probs = counts.groupby(level=0).transform(lambda x: x / x.sum())
    
    # Calculate entropy: -sum(p * log(p)) for each group
    entropy = -(probs * np.log(probs + 1e-9)).groupby(level=0).sum()
    
    return entropy


def pct_amount(df, flag_col, amount_col='payment_amount'):
    """
    Calculate the fraction of total donation amount coming from flagged rows.
    
    Example: If flag_col='daf_payment', returns what % of each donor's
    total giving came through DAF donations.
    
    Parameters
    ----------
    df : pd.DataFrame
        Donation records with donor_id column
    flag_col : str
        Name of binary indicator column (1 = flagged, 0 = not flagged)
    amount_col : str, default='payment_amount'
        Name of amount column to aggregate
        
    Returns
    -------
    pct : pd.Series
        Fraction of amount from flagged rows, indexed by donor_id
    """

    if df.empty:
        return pd.Series(dtype=float)
    
    # Filter THEN group (vectorized, no lambda)
    flagged_df = df[df[flag_col] == 1]
    amt_flag = flagged_df.groupby('donor_id')[amount_col].sum()
    amt_total = df.groupby('donor_id')[amount_col].sum()
    amt_flag = amt_flag.reindex(amt_total.index, fill_value=0)
    
    # Return fraction (with small epsilon to avoid division by zero)
    return amt_flag / (amt_total + 1e-6)


def pct_count(df, flag_col):
    """
    Calculate the fraction of total gift count coming from flagged rows.
    
    Example: If flag_col='gift_card_purchase', returns what % of each donor's
    gifts were gift card purchases.
    
    Parameters
    ----------
    df : pd.DataFrame
        Donation records with donor_id column
    flag_col : str
        Name of binary indicator column
        
    Returns
    -------
    pct : pd.Series
        Fraction of gifts that are flagged, indexed by donor_id
    """
    if df.empty:
        return pd.Series(dtype=float)
        
    # Filter THEN group (vectorized)
    flagged_df = df[df[flag_col] == 1]
    cnt_flag = flagged_df.groupby('donor_id').size()
    cnt_total = df.groupby('donor_id').size()
    cnt_flag = cnt_flag.reindex(cnt_total.index, fill_value=0)
    
    return cnt_flag / (cnt_total + 1e-6)

## 2. Feature Group Functions

Each function builds a logical group of related features.

### 2.1 Identity & Demographics

In [3]:
def _identity_and_zip_features(dpr_pre_T, df_zip_acs, md_pre_T=None):
    """
    Build identity features and ZIP-level demographics.
    
    Parameters:
    -----------
    dpr_pre_T : pd.DataFrame
        Donation records before T
    df_zip_acs : pd.DataFrame
        ZIP-level ACS demographics
    md_pre_T : pd.DataFrame, optional
        Monthly donation data (for ZIP fallback for monthly-only donors)
    """
    # Create base index from all possible donors
    all_donor_ids = dpr_pre_T['donor_id'].unique()
    if md_pre_T is not None:
        all_donor_ids = pd.Index(
            pd.unique(
                pd.concat([
                    pd.Series(dpr_pre_T['donor_id'].unique()),
                    pd.Series(md_pre_T['donor_id'].unique())
                ])
            )
        )
    
    if dpr_pre_T.empty and (md_pre_T is None or md_pre_T.empty):
        return pd.DataFrame(index=pd.Index([], name='donor_id'))

    # Get ZIP from PROJECT donations (if available)
    if not dpr_pre_T.empty:
        donor_zip5_from_dpr = (
            dpr_pre_T
            .sort_values('payment_date')
            .groupby('donor_id')['donor_zip']
            .first()
        )
    else:
        donor_zip5_from_dpr = pd.Series(dtype=object, name='donor_zip')
    
    # Get ZIP from MONTHLY data as fallback
    if md_pre_T is not None and not md_pre_T.empty and 'donor_zip' in md_pre_T.columns:
        donor_zip5_from_monthly = (
            md_pre_T
            .sort_values('monthly_subscription_joined_date')
            .groupby('donor_id')['donor_zip']
            .first()
        )
        # Combine: use dpr ZIP if available, otherwise monthly ZIP
        donor_zip5_raw = donor_zip5_from_dpr.combine_first(donor_zip5_from_monthly)
    else:
        donor_zip5_raw = donor_zip5_from_dpr

    # Binary status flags (from dpr_pre_T)
    if not dpr_pre_T.empty:
        is_teacher = dpr_pre_T.groupby('donor_id')['is_teacher'].max()
        is_teacher_referred = dpr_pre_T.groupby('donor_id')['is_teacher_referred'].max()
        is_marketing_subscribed = dpr_pre_T.groupby('donor_id')['subscribed_to_marketing_emails'].max()
        is_major_gift_donor = dpr_pre_T.groupby('donor_id')['major_gift_donor'].max()
        
        # Account credit
        ever_used_account_credit = (
            dpr_pre_T.groupby('donor_id')['account_credit_balance']
            .max()
            .gt(0)
            .astype(int)
        )
        current_account_credit_balance = (
            dpr_pre_T
            .sort_values('payment_date')
            .groupby('donor_id')['account_credit_balance']
            .last()
        )
    else:
        # Create empty series if no dpr data
        is_teacher = pd.Series(dtype=float)
        is_teacher_referred = pd.Series(dtype=float)
        is_marketing_subscribed = pd.Series(dtype=float)
        is_major_gift_donor = pd.Series(dtype=float)
        ever_used_account_credit = pd.Series(dtype=int)
        current_account_credit_balance = pd.Series(dtype=float)

    # --- Donor Type Flags (always create for all donors) ---
    # Determine which donors appear in each data source
    project_donor_ids = set(dpr_pre_T['donor_id'].unique()) if not dpr_pre_T.empty else set()
    monthly_donor_ids = set(md_pre_T['donor_id'].unique()) if md_pre_T is not None and not md_pre_T.empty else set()
    
    # Create flags for all donors in our universe
    has_project_history = pd.Series([d in project_donor_ids for d in all_donor_ids], index=all_donor_ids, dtype=int)
    has_monthly_history = pd.Series([d in monthly_donor_ids for d in all_donor_ids], index=all_donor_ids, dtype=int)
    
    # Composite flags for donor segmentation
    is_monthly_only_donor = ((has_monthly_history == 1) & (has_project_history == 0)).astype(int)
    is_project_only_donor = ((has_project_history == 1) & (has_monthly_history == 0)).astype(int)
    is_hybrid_donor = ((has_project_history == 1) & (has_monthly_history == 1)).astype(int)

    # --- ZIP / ACS JOIN (dtype-safe) ------------------------------------
    donor_zip5 = pd.to_numeric(donor_zip5_raw, errors="coerce")

    df_zip_acs = df_zip_acs.copy()
    if 'ZIP5' in df_zip_acs.columns:
        df_zip_acs = df_zip_acs.set_index('ZIP5')
    df_zip_acs.index = pd.to_numeric(df_zip_acs.index, errors="coerce")

    acs = donor_zip5.to_frame('donor_zip5').join(
        df_zip_acs,
        on='donor_zip5',
        how='left'
    )

    # Assemble output
    out = pd.DataFrame(index=donor_zip5.index)
    out['donor_zip5'] = donor_zip5
    out['has_project_history'] = has_project_history
    out['has_monthly_history'] = has_monthly_history
    out['is_monthly_only_donor'] = is_monthly_only_donor
    out['is_project_only_donor'] = is_project_only_donor
    out['is_hybrid_donor'] = is_hybrid_donor
    out['is_teacher'] = is_teacher
    out['is_teacher_referred'] = is_teacher_referred
    out['is_marketing_subscribed'] = is_marketing_subscribed
    out['is_major_gift_donor'] = is_major_gift_donor
    out['ever_used_account_credit'] = ever_used_account_credit
    out['current_account_credit_balance'] = current_account_credit_balance

    # ACS features
    out['zip_pct_households_with_children'] = acs['pct_households_with_children']
    out['zip_pct_in_labor_force'] = acs['pct_in_labor_force']
    out['zip_unemployment_rate'] = acs['unemployment_rate']
    out['zip_pct_single_parent'] = acs['pct_single_parent']
    out['zip_pct_minority'] = acs['pct_minority']
    out['zip_avg_household_size'] = acs['avg_household_size']
    out['zip_median_age'] = acs['median_age']
    out['zip_median_home_value'] = acs['median_home_value']
    
    if 'total_population' in acs.columns:
        out['zip_log_total_population'] = np.log1p(acs['total_population'])
    elif 'log_total_population' in acs.columns:
        out['zip_log_total_population'] = acs['log_total_population']
    else:
        out['zip_log_total_population'] = np.nan
        
    return out

### 2.2 Lifetime Giving Behavior

In [4]:
def _lifetime_giving_features(dpr_pre_T):
    """
    Build cumulative giving features using all donations before T.
    
    Features created:
    - first_donation_date, last_donation_date: Temporal boundaries
    - tenure_days, tenure_years, tenure_bucket: How long donor has been active
    - lifetime_gift_count, lifetime_amount: Total volume
    - lifetime_median/max/cv_gift_amount: Distribution of gift sizes
    - mean/cv_gap_between_gifts_days: Giving rhythm/consistency
    - max_donation_sequence_number: How many gifts total
    - pct_early_gifts_in_lifetime: Concentration in first 3 gifts
    
    Parameters
    ----------
    dpr_pre_T : pd.DataFrame
        Donor Project Records filtered to payment_date < T
        
    Returns
    -------
    features : pd.DataFrame
        Feature matrix indexed by donor_id
    """
    if dpr_pre_T.empty:
        return pd.DataFrame(index=pd.Index([], name='donor_id'))
    
    dpr = dpr_pre_T.copy()
    dpr = dpr.sort_values(['donor_id', 'payment_date'])
    
    # --- Temporal boundaries ---
    first_donation_date = dpr.groupby('donor_id')['payment_date'].min()
    last_donation_date = dpr.groupby('donor_id')['payment_date'].max()
    
    # Tenure: time from first to last donation
    # (Note: this is OBSERVED tenure up to T, not time since first donation to T)
    tenure_days = (last_donation_date - first_donation_date).dt.days
    tenure_years = tenure_days / 365.25
    
    # Bucket tenure for non-linear patterns (categorical)
    tenure_bucket_cat = pd.cut(
        tenure_years,
        bins=[0, 1, 3, 5, 100],
        labels=['<1y', '1-3y', '3-5y', '5y+'],
        right=False
    )
    
    # Map bucket to lower-bound in YEARS as float64
    tenure_bucket = tenure_bucket_cat.map({
        '<1y': 0.0,
        '1-3y': 1.0,
        '3-5y': 3.0,
        '5y+': 5.0,
    }).astype(float)
    
    # --- Volume metrics ---
    lifetime_gift_count = dpr.groupby('donor_id')['payment_amount'].size()
    lifetime_amount = dpr.groupby('donor_id')['payment_amount'].sum()
    
    # --- Distribution of gift sizes ---
    gifts_by_donor = dpr.groupby('donor_id')['payment_amount']
    med = gifts_by_donor.median()
    mx = gifts_by_donor.max()
    mean = gifts_by_donor.mean()
    std = gifts_by_donor.std()
    # Coefficient of variation: std/mean
    # High CV = erratic giving amounts, Low CV = consistent amounts
    cv = std / (mean + 1e-9)
    
    # --- Inter-gift timing patterns ---
    # Calculate days between consecutive gifts
    dpr['prev_payment_date'] = dpr.groupby('donor_id')['payment_date'].shift(1)
    dpr['gap_days'] = (dpr['payment_date'] - dpr['prev_payment_date']).dt.days
    
    gap_by_donor = dpr.groupby('donor_id')['gap_days']
    gap_mean = gap_by_donor.mean()
    gap_std = gap_by_donor.std()
    # CV of gaps: High = irregular giving, Low = regular/predictable
    cv_gap = gap_std / (gap_mean + 1e-9)
    
    # --- Gift sequence metrics ---
    # donation_n = sequential gift number for this donor
    max_donation_sequence_number = dpr.groupby('donor_id')['donation_n'].max()
    
    # What fraction of gifts were in the "early stage" (first 3 gifts)?
    # High concentration here might indicate acquisition success but poor retention
    dpr['is_early_gift'] = dpr['donation_n'] <= 3
    pct_early_gifts = dpr.groupby('donor_id')['is_early_gift'].mean()
    
    # --- Assemble output ---
    out = pd.DataFrame(index=lifetime_amount.index)
    
    out['first_donation_date'] = first_donation_date
    out['last_donation_date'] = last_donation_date
    out['tenure_days'] = tenure_days
    out['tenure_years'] = tenure_years
    out['tenure_bucket'] = tenure_bucket

    
    out['lifetime_gift_count'] = lifetime_gift_count
    out['lifetime_amount'] = lifetime_amount
    out['lifetime_median_gift_amount'] = med
    out['lifetime_max_gift_amount'] = mx
    out['lifetime_cv_gift_amount'] = cv
    
    out['mean_gap_between_gifts_days'] = gap_mean
    out['cv_gap_between_gifts_days'] = cv_gap
    
    out['max_donation_sequence_number'] = max_donation_sequence_number
    out['pct_early_gifts_in_lifetime'] = pct_early_gifts
    
    return out

### 2.3 Windowed Giving & Velocity

These features capture recent activity and trends.

In [5]:
def _windowed_giving_features(dpr_pre_T, dpr_3m, dpr_12m, dpr_36m, dpr_3to12m, dpr_12to36m, T):
    """
    Build windowed giving features and velocity metrics.
    
    Features created:
    - gift_count/amount/median_amount for 3m, 12m, 36m windows
    - days_since_last_gift, days_since_second_to_last_gift: Recency
    - amount_velocity_0to3_vs_3to12: Is giving accelerating or decelerating?
    - amount_velocity_0to12_vs_12to36: Longer-term trend
    - count_velocity_0to12_vs_12to36: Frequency trend
    
    Velocity > 1 indicates acceleration (more recent activity)
    Velocity < 1 indicates deceleration (declining activity)
    
    Parameters
    ----------
    dpr_pre_T : pd.DataFrame
        All donations before T
    dpr_3m, dpr_12m, dpr_36m : pd.DataFrame
        Donations in respective windows
    dpr_3to12m, dpr_12to36m : pd.DataFrame
        Intermediate periods for velocity calculations
    T : pd.Timestamp
        Reference time
        
    Returns
    -------
    features : pd.DataFrame
        Feature matrix indexed by donor_id
    """
    out_index = dpr_pre_T['donor_id'].unique()
    out = pd.DataFrame(index=out_index)
    
    # --- 3-month window (recent/short-term) ---
    if not dpr_3m.empty:
        by_3m = dpr_3m.groupby('donor_id')['payment_amount']
        out['gift_count_3m'] = by_3m.size()
        out['gift_amount_3m'] = by_3m.sum()
        out['median_gift_amount_3m'] = by_3m.median()
    
    # --- 12-month window (mid-term) ---
    if not dpr_12m.empty:
        by_12m = dpr_12m.groupby('donor_id')['payment_amount']
        out['gift_count_12m'] = by_12m.size()
        out['gift_amount_12m'] = by_12m.sum()
        out['median_gift_amount_12m'] = by_12m.median()
    
    # --- 36-month window (long-term) ---
    if not dpr_36m.empty:
        by_36m = dpr_36m.groupby('donor_id')['payment_amount']
        out['gift_count_36m'] = by_36m.size()
        out['gift_amount_36m'] = by_36m.sum()
        out['median_gift_amount_36m'] = by_36m.median()
    
    # --- Recency: days since last activity ---
    last_gift_date = dpr_pre_T.groupby('donor_id')['payment_date'].max()
    out['days_since_last_gift'] = (T - last_gift_date).dt.days
    
    # Second-to-last gift: helps detect if donor is becoming less frequent
    # Sort by payment_date descending and use groupby.nth(1)
    second_last = (
        dpr_pre_T
        .sort_values(['donor_id', 'payment_date'], ascending=[True, False])
        .groupby('donor_id')['payment_date']
        .nth(1)  # second row per donor, or NaT if fewer than 2
    )
    
    out['days_since_second_to_last_gift'] = (T - second_last).dt.days
    
    # --- Velocity metrics: are donations accelerating or decelerating? ---
    
    # Recent 3m vs prior 9m (months 3-12)
    amt_0_3 = out.get('gift_amount_3m', pd.Series(0, index=out.index))
    amt_3_12 = (
        dpr_3to12m.groupby('donor_id')['payment_amount'].sum()
        if not dpr_3to12m.empty
        else pd.Series(0, index=out.index)
    )
    # Ratio > 1 means recent giving exceeds prior period
    out['amount_velocity_0to3_vs_3to12'] = amt_0_3 / (amt_3_12 + 1e-6)
    
    # Recent 12m vs prior 24m (months 12-36)
    amt_0_12 = out.get('gift_amount_12m', pd.Series(0, index=out.index))
    amt_12_36 = (
        dpr_12to36m.groupby('donor_id')['payment_amount'].sum()
        if not dpr_12to36m.empty
        else pd.Series(0, index=out.index)
    )
    out['amount_velocity_0to12_vs_12to36'] = amt_0_12 / (amt_12_36 + 1e-6)
    
    # Gift frequency velocity (count-based)
    cnt_0_12 = out.get('gift_count_12m', pd.Series(0, index=out.index))
    cnt_12_36 = (
        dpr_12to36m.groupby('donor_id')['payment_amount'].size()
        if not dpr_12to36m.empty
        else pd.Series(0, index=out.index)
    )
    out['count_velocity_0to12_vs_12to36'] = cnt_0_12 / (cnt_12_36 + 1e-6)
    
    return out

### 2.4 Channel & Payment Type Mix

How donors give: DAF, green payments, gift cards, etc.

In [6]:
def _channel_mix_features(dpr_pre_T, dpr_12m):
    """
    Build features describing how donors give (payment methods, channels).
    
    For each channel, we compute both lifetime and 12-month versions:
    - pct_amount_*: What fraction of $ came through this channel?
    - pct_count_*: What fraction of gifts came through this channel?
    
    Channels covered:
    - DAF (Donor Advised Fund) payments
    - Green payments (environmental offset donations)
    - Gift card purchases
    - Big event donations (e.g., giving days)
    - Optional donation behavior
    - Anonymous gifts
    - Classroom essentials projects
    
    Parameters
    ----------
    dpr_pre_T : pd.DataFrame
        All donations before T (for lifetime metrics)
    dpr_12m : pd.DataFrame
        Donations in last 12 months (for recent behavior)
        
    Returns
    -------
    features : pd.DataFrame
        Feature matrix indexed by donor_id
    """
    out_index = dpr_pre_T['donor_id'].unique()
    out = pd.DataFrame(index=out_index)
    
    # --- DAF donations ---
    # DAF = Donor Advised Fund, typically indicates sophisticated/high-capacity donors
    out['pct_amount_daf_lifetime'] = pct_amount(dpr_pre_T, 'daf_payment')
    out['pct_count_daf_lifetime'] = pct_count(dpr_pre_T, 'daf_payment')
    out['pct_amount_daf_12m'] = pct_amount(dpr_12m, 'daf_payment')
    out['pct_count_daf_12m'] = pct_count(dpr_12m, 'daf_payment')
    
    # --- Green payments ---
    # Optional environmental offset donations
    if 'green_payment_amount' in dpr_pre_T.columns:
        by_life = dpr_pre_T.groupby('donor_id')
        by_12 = dpr_12m.groupby('donor_id')
        
        out['pct_amount_green_lifetime'] = (
            by_life['green_payment_amount'].sum()
            / (by_life['payment_amount'].sum() + 1e-6)
        )
        out['pct_amount_green_12m'] = (
            by_12['green_payment_amount'].sum()
            / (by_12['payment_amount'].sum() + 1e-6)
        )
        out['pct_count_green_lifetime'] = pct_count(dpr_pre_T, 'is_green_payment')
        out['pct_count_green_12m'] = pct_count(dpr_12m, 'is_green_payment')
    
    # --- Gift card purchases ---
    # Donors buying gift cards to give to others
    out['pct_gifts_gift_card_lifetime'] = pct_count(dpr_pre_T, 'gift_card_purchase')
    out['pct_amount_gift_card_lifetime'] = pct_amount(dpr_pre_T, 'gift_card_purchase')
    out['pct_gifts_gift_card_12m'] = pct_count(dpr_12m, 'gift_card_purchase')
    out['pct_amount_gift_card_12m'] = pct_amount(dpr_12m, 'gift_card_purchase')
    
    # --- Big event donations ---
    # Giving days, campaigns, etc.
    out['pct_amount_big_event_lifetime'] = pct_amount(dpr_pre_T, 'payment_on_big_event')
    out['pct_count_big_event_lifetime'] = pct_count(dpr_pre_T, 'payment_on_big_event')
    out['pct_amount_big_event_12m'] = pct_amount(dpr_12m, 'payment_on_big_event')
    out['pct_count_big_event_12m'] = pct_count(dpr_12m, 'payment_on_big_event')
    
    # --- Optional donation rate ---
    # When donors can add optional amounts (e.g., to cover fees)
    out['avg_optional_donation_rate_lifetime'] = (
        dpr_pre_T.groupby('donor_id')['optional_donation_rate'].mean()
    )
    out['avg_optional_donation_rate_12m'] = (
        dpr_12m.groupby('donor_id')['optional_donation_rate'].mean()
    )
    
    # --- Anonymous donations ---
    # Privacy-conscious or humble donors
    out['pct_gifts_anonymous_lifetime'] = pct_count(dpr_pre_T, 'donation_is_anonymous')
    out['pct_gifts_anonymous_12m'] = pct_count(dpr_12m, 'donation_is_anonymous')
    
    # --- Classroom essentials ---
    # Donations to specific project type (basic supplies)
    out['pct_amount_classroom_essentials_lifetime'] = pct_amount(
        dpr_pre_T, 'is_classroom_essentials_list'
    )
    out['pct_amount_classroom_essentials_12m'] = pct_amount(
        dpr_12m, 'is_classroom_essentials_list'
    )
    
    return out

### 2.5 Monthly Giving Program

Subscription/recurring donation features.

In [7]:
def _monthly_features(md_pre_T, dpr_pre_T, dpr_12m, T):
    """
    Build monthly subscription program features.
    
    Features created:
    - is_monthly_donor_current: Currently subscribed?
    - monthly_lifetime_amount, monthly_amount_12m: How much via subscription
    - monthly_median_gift_amount: Typical subscription size
    - pct_amount_monthly_*: What fraction of total giving is subscription
    - months_on_program, months_since_last_monthly_charge: Tenure metrics
    - monthly_longest_streak_months: Longest uninterrupted run
    - monthly_joined_before_first_project_gift: Acquisition source indicator
    
    Parameters
    ----------
    md_pre_T : pd.DataFrame
        Monthly subscription records where join date < T
    dpr_pre_T : pd.DataFrame
        All donations before T (for computing fractions)
    dpr_12m : pd.DataFrame
        Donations in last 12 months
    T : pd.Timestamp
        Reference time
        
    Returns
    -------
    features : pd.DataFrame
        Feature matrix indexed by donor_id
    """
    if md_pre_T.empty:
        return pd.DataFrame(index=dpr_pre_T['donor_id'].unique())
    
    out_index = pd.Index(md_pre_T['donor_id'].unique(), name='donor_id')
    out = pd.DataFrame(index=out_index)
    
    # --- Current subscription status ---
    def is_active(row):
        """Check if subscription is active at time T"""
        retired = row['monthly_subscription_retired_date']
        return (
            (row['monthly_subscription_joined_date'] <= T) and
            (pd.isna(retired) or (retired > T))
        )
    
    # Vectorized boolean operations (processes all rows at once)
    md_pre_T['is_active'] = (
        (md_pre_T['monthly_subscription_joined_date'] <= T) &
        (md_pre_T['monthly_subscription_retired_date'].isna() | 
         (md_pre_T['monthly_subscription_retired_date'] > T))
    )
    
    out['is_monthly_donor_current'] = md_pre_T.groupby('donor_id')['is_active'].max().astype(float)
    
    # --- Lifetime monthly amounts ---
    by_life = md_pre_T.groupby('donor_id')
    out['monthly_lifetime_amount'] = by_life['monthly_subscription_payment_amount'].sum()
    out['monthly_median_gift_amount'] = by_life['monthly_subscription_payment_amount'].median()
    
    # --- 12-month window ---
    if 'charge_date' in md_pre_T.columns:
        md_12m = md_pre_T[
            (md_pre_T['charge_date'] >= T - pd.DateOffset(months=12)) &
            (md_pre_T['charge_date'] < T)
        ]
        out['monthly_amount_12m'] = (
            md_12m.groupby('donor_id')['monthly_subscription_payment_amount'].sum()
        )
    
    # --- Fraction of total giving that's monthly ---
    # This shows how dependent a donor is on subscription vs one-time gifts
    total_life = dpr_pre_T.groupby('donor_id')['payment_amount'].sum()
    total_12 = dpr_12m.groupby('donor_id')['payment_amount'].sum()
    
    out['pct_amount_monthly_lifetime'] = (
        out['monthly_lifetime_amount'] / (total_life + 1e-6)
    )
    out['pct_amount_monthly_12m'] = (
        out.get('monthly_amount_12m', 0) / (total_12 + 1e-6)
    )
    
    # --- Months on program ---
    # How long has donor been (or was) subscribed?
    def months_on_program_fn(x):
        join = x['monthly_subscription_joined_date'].min()
        retire = x['monthly_subscription_retired_date'].dropna().min()
        # If still active, use T; otherwise use retirement date
        end = min(T, retire) if pd.notna(retire) else T
        return (end.to_period('M') - join.to_period('M')).n
    
    # Vectorized date operations
    join_dates = md_pre_T.groupby('donor_id')['monthly_subscription_joined_date'].min()
    retire_dates = md_pre_T.groupby('donor_id')['monthly_subscription_retired_date'].min()
    end_dates = retire_dates.fillna(T).clip(upper=T)
    out['months_on_program'] = (end_dates.dt.to_period('M') - join_dates.dt.to_period('M')).apply(lambda x: x.n)

    # --- Recency of last charge ---
    if 'charge_date' in md_pre_T.columns:
        last_charge = md_pre_T.groupby('donor_id')['charge_date'].max()
        out['months_since_last_monthly_charge'] = (
            (T.to_period('M') - last_charge.dt.to_period('M')).astype('int')
        )
    
    # --- Longest streak ---
    # Maximum consecutive months of successful charges
    out['monthly_longest_streak_months'] = (
        md_pre_T.groupby('donor_id')['monthly_subscription_longest_streak'].max()
    )
    
    # --- Acquisition indicator ---
    # Did donor join monthly program BEFORE making first project donation?
    first_monthly_join = (
        md_pre_T.groupby('donor_id')['monthly_subscription_joined_date'].min()
    )
    first_donation_date = (
        dpr_pre_T.groupby('donor_id')['payment_date'].min()
    )

    # Align on union of donor_ids
    idx_union = first_monthly_join.index.union(first_donation_date.index)
    fmj = first_monthly_join.reindex(idx_union)
    fdd = first_donation_date.reindex(idx_union)

    joined_before = (fmj < fdd)
    joined_before = joined_before.fillna(False).astype(int)

    out['monthly_joined_before_first_project_gift'] = (
        joined_before.reindex(out.index).fillna(0).astype(int)
    )

    return out

### 2.6 Teacher, School & Content Preferences

Loyalty vs diversification patterns.

In [8]:
def _teacher_school_features(dpr_pre_T):
    """
    Build features about donor loyalty/diversification patterns.
    
    For each dimension (teacher, school, category, grade), we compute:
    - entropy_*: How scattered/diverse is giving? (0=single focus, higher=broad)
    - num_unique_*: How many different entities supported?
    - pct_amount_to_top_*: Concentration on favorite (0.5 = half to one teacher)
    
    Also includes:
    - first/last project behavior
    - average teacher performance metrics
    
    High entropy + low top% = broad, exploratory giving
    Low entropy + high top% = loyal, focused giving
    
    Parameters
    ----------
    dpr_pre_T : pd.DataFrame
        All donations before T
        
    Returns
    -------
    features : pd.DataFrame
        Feature matrix indexed by donor_id
    """
    if dpr_pre_T.empty:
        return pd.DataFrame(index=pd.Index([], name='donor_id'))
    
    dpr = dpr_pre_T.copy()
    out_index = dpr['donor_id'].unique()
    out = pd.DataFrame(index=out_index)
    
    # --- Entropy (diversity) metrics ---
    # Higher entropy = more diverse/scattered giving
    if not dpr_pre_T.empty:
        out['entropy_teacher'] = entropy_vectorized(dpr_pre_T, 'donor_id', 'teacher_id')
        out['entropy_school'] = entropy_vectorized(dpr_pre_T, 'donor_id', 'school_id')
        out['entropy_zip'] = entropy_vectorized(dpr_pre_T, 'donor_id', 'school_zip')
        out['entropy_category'] = entropy_vectorized(dpr_pre_T, 'donor_id', 'project_category')
        out['entropy_grade'] = entropy_vectorized(dpr_pre_T, 'donor_id', 'project_grade')
    else:
        out['entropy_teacher'] = np.nan
        out['entropy_school'] = np.nan
        out['entropy_zip'] = np.nan
        out['entropy_category'] = np.nan
        out['entropy_grade'] = np.nan
        
    # --- Unique entity counts ---
    out['num_unique_teachers'] = dpr.groupby('donor_id')['teacher_id'].nunique()
    out['num_unique_schools'] = dpr.groupby('donor_id')['school_id'].nunique()
    out['num_unique_school_zips'] = dpr.groupby('donor_id')['school_zip'].nunique()
    out['num_unique_categories'] = dpr.groupby('donor_id')['project_category'].nunique()
    out['num_unique_grades'] = dpr.groupby('donor_id')['project_grade'].nunique()
    
    # --- Concentration on "favorite" entity ---
    def pct_amount_to_top(dpr, entity_col):
        """
        For each donor, what fraction of total payment_amount went to the single
        most-supported entity in `entity_col`?
        """
        amt_by_entity = (
            dpr.groupby(['donor_id', entity_col])['payment_amount']
            .sum()
        )
        stats = amt_by_entity.groupby('donor_id').agg(['max', 'sum'])
        return stats['max'] / (stats['sum'] + 1e-6)
    
    out['pct_amount_to_top_teacher'] = (
        pct_amount_to_top(dpr, 'teacher_id').reindex(out.index)
    )
    out['pct_amount_to_top_school'] = (
        pct_amount_to_top(dpr, 'school_id').reindex(out.index)
    )
    out['pct_amount_to_top_category'] = (
        pct_amount_to_top(dpr, 'project_category').reindex(out.index)
    )
    out['pct_amount_to_top_grade'] = (
        pct_amount_to_top(dpr, 'project_grade').reindex(out.index)
    )
    
    # --- First/last project role ---
    # These flags indicate timing within project lifecycle
    dpr['is_first'] = dpr['gift_is_projects_first'] == 1
    dpr['is_last'] = dpr['gift_is_projects_last'] == 1
    
    out['pct_gifts_first_project'] = dpr.groupby('donor_id')['is_first'].mean()
    out['pct_gifts_last_project'] = dpr.groupby('donor_id')['is_last'].mean()
    
    # --- Teacher performance exposure ---
    # What's the typical "quality" of teachers this donor supports?
    teacher_perf = dpr.groupby(['donor_id', 'teacher_id']).agg({
        'teacher_lifetime_projects_fully_funded': 'max',
        'teacher_lifetime_donations': 'max'
    }).reset_index()
    
    out['mean_teacher_lifetime_projects_fully_funded'] = (
        teacher_perf.groupby('donor_id')['teacher_lifetime_projects_fully_funded'].mean()
    )
    out['mean_teacher_lifetime_donations'] = (
        teacher_perf.groupby('donor_id')['teacher_lifetime_donations'].mean()
    )
    
    return out

### 2.7 Seasonality Features

In [9]:
def _seasonality_features(dpr_pre_T):
    """
    Build features capturing seasonal giving patterns.
    
    Features created:
    - pct_amount_in_back_to_school: Aug-Sep concentration
    - pct_amount_in_final_week_of_year: Year-end giving surge
    - pct_amount_on_weekends: Day-of-week preference
    - entropy_gift_month: How scattered giving is across calendar year
    - pct_amount_in_top_month/quarter: Concentration in peak period
    - first_donation_month/quarter/dow: Acquisition seasonality
    
    Parameters
    ----------
    dpr_pre_T : pd.DataFrame
        All donations before T
        
    Returns
    -------
    features : pd.DataFrame
        Feature matrix indexed by donor_id
    """
    if dpr_pre_T.empty:
        return pd.DataFrame(index=pd.Index([], name='donor_id'))
    
    dpr = dpr_pre_T.copy()
    
    # Extract time components
    dpr['month'] = dpr['payment_date'].dt.month
    dpr['quarter'] = dpr['payment_date'].dt.quarter
    dpr['dow'] = dpr['payment_date'].dt.dayofweek  # 0=Monday
    dpr['dayofyear'] = dpr['payment_date'].dt.dayofyear
    
    by_donor = dpr.groupby('donor_id')
    
    # --- Helper for amount-based fractions ---
    def frac_amount_where(mask):
        """Return function that computes fraction of $ in masked rows"""
        def f(g):
            amt_mask = g.loc[mask(g), 'payment_amount'].sum()
            return amt_mask / (g['payment_amount'].sum() + 1e-6)
        return f
    
    # --- Specific seasonal patterns ---
    # Back-to-school (August-September)
    bts_mask = lambda g: g['month'].isin([8, 9])
    pct_amount_in_back_to_school = by_donor.apply(frac_amount_where(bts_mask))
    
    # Year-end giving (last ~week, day 359+)
    final_mask = lambda g: g['dayofyear'] >= 359
    pct_amount_in_final_week_of_year = by_donor.apply(frac_amount_where(final_mask))
    
    # Weekend giving (Sat/Sun = 5/6)
    weekend_mask = lambda g: g['dow'].isin([5, 6])
    pct_amount_on_weekends = by_donor.apply(frac_amount_where(weekend_mask))
    
    # --- Overall calendar dispersion ---
    # High entropy = gifts spread throughout year
    # Low entropy = concentrated in specific months
    entropy_gift_month = entropy_vectorized(dpr, 'donor_id', 'month')
    
    # --- Peak period concentration ---
    def pct_in_top_period(group, col):
        """Fraction of $ in the single highest-giving period"""
        amt = group.groupby(col)['payment_amount'].sum()
        return amt.max() / (amt.sum() + 1e-6)
    
    pct_amount_in_top_month = by_donor.apply(lambda g: pct_in_top_period(g, 'month'))
    pct_amount_in_top_quarter = by_donor.apply(lambda g: pct_in_top_period(g, 'quarter'))
    
    # --- First-donation seasonality (STATIC) ---
    # These capture when donor was acquired
    first_dates = by_donor['payment_date'].min()
    first_month = first_dates.dt.month
    first_quarter = first_dates.dt.quarter
    first_dow = first_dates.dt.dayofweek
    
    # Cyclic encoding for day-of-week (preserves circular nature)
    first_dow_sin = np.sin(2 * np.pi * first_dow / 7)
    first_dow_cos = np.cos(2 * np.pi * first_dow / 7)
    
    # --- Assemble output ---
    out = pd.DataFrame(index=first_dates.index)
    
    out['pct_amount_in_back_to_school'] = pct_amount_in_back_to_school
    out['pct_amount_in_final_week_of_year'] = pct_amount_in_final_week_of_year
    out['pct_amount_on_weekends'] = pct_amount_on_weekends
    out['entropy_gift_month'] = entropy_gift_month
    out['pct_amount_in_top_month'] = pct_amount_in_top_month
    out['pct_amount_in_top_quarter'] = pct_amount_in_top_quarter
    
    out['first_donation_month'] = first_month
    out['first_donation_quarter'] = first_quarter
    out['first_donation_dow_sin'] = first_dow_sin
    out['first_donation_dow_cos'] = first_dow_cos
    
    return out

### 2.8 Email Engagement

In [10]:
def _email_features(email_pre_T, email_3m, email_12m, T):
    """
    Build email engagement features from 12-month email summary data.
    
    Features created:
    - emails_sent/opened/clicked for 3m and 12m windows
    - open_rate, click_rate (clicks per email sent)
    - email_open_rate_velocity: recent vs longer-term trend
    - days_since_last_email_sent: recency
    
    Note: This uses monthly aggregated data, so recency is approximate
    
    Parameters
    ----------
    email_pre_T : pd.DataFrame
        All email events before T
    email_3m, email_12m : pd.DataFrame
        Email events in respective windows
    T : pd.Timestamp
        Reference time
        
    Returns
    -------
    features : pd.DataFrame
        Feature matrix indexed by donor_id
    """
    if email_pre_T.empty:
        return pd.DataFrame(index=pd.Index([], name='donor_id'))
    
    def email_agg(df):
        """Aggregate email metrics for a given window"""
        if df.empty:
            idx = email_pre_T['donor_id'].unique()
            zero = pd.Series(0, index=idx)
            return zero, zero, zero, zero, zero
        
        by = df.groupby('donor_id')
        sent = by['email_sent_count'].sum()
        opened = by['email_open_count'].sum()
        clicked = by['email_click_count'].sum()
        
        # Rates: opens/clicks per email sent
        open_rate = opened / (sent + 1e-6)
        click_rate = clicked / (sent + 1e-6)
        
        return sent, opened, clicked, open_rate, click_rate
    
    # Aggregate for both windows
    sent_3, open_3, click_3, or_3, cr_3 = email_agg(email_3m)
    sent_12, open_12, click_12, or_12, cr_12 = email_agg(email_12m)
    
    idx = email_pre_T['donor_id'].unique()
    out = pd.DataFrame(index=idx)
    
    # 3-month window
    out['emails_sent_3m'] = sent_3
    out['emails_opened_3m'] = open_3
    out['emails_clicked_3m'] = click_3
    out['email_open_rate_3m'] = or_3
    out['email_click_rate_3m'] = cr_3
    
    # 12-month window
    out['emails_sent_12m'] = sent_12
    out['emails_opened_12m'] = open_12
    out['emails_clicked_12m'] = click_12
    out['email_open_rate_12m'] = or_12
    out['email_click_rate_12m'] = cr_12
    
    # Velocity: is engagement improving or declining?
    # Positive = recent engagement higher than long-term average
    out['email_open_rate_velocity_3m_vs_12m'] = or_3 - or_12
    
    # Recency (approximate, since data is monthly)
    last_email_month = email_pre_T.groupby('donor_id')['email_month_start'].max()
    out['days_since_last_email_sent'] = (T - last_email_month).dt.days
    
    # Note: email type mix features would go here if you have
    # a mapping from email_type to type_group (appeal, newsletter, etc.)
    
    return out

### 2.9 Site Behavior

In [11]:
def _site_features(site_pre_T, site_3m, T):
    """
    Build on-site browsing and engagement features.
    
    Features created:
    - days_with_any_site_activity_3m: Active days count
    - avg_sessions_per_active_day_3m: Session frequency
    - avg_session_duration_min_3m: Session length
    - checkout_intent_min_per_session_3m: Cart engagement
    - days_since_last_cart_visit: Browse-to-buy recency (relative to T)
    - campaign_session_share_3m: Attribution
    - share_*_page_session_pct_3m: Page type mix
    - device_share_*_3m: Device usage profile
    
    Parameters
    ----------
    site_pre_T : pd.DataFrame
        Site events with activity_date < T
    site_3m : pd.DataFrame
        Site events with activity_date in [T-3m, T)
    T : pd.Timestamp
        As-of timestamp
    """
    if site_pre_T.empty:
        return pd.DataFrame(index=pd.Index([], name='donor_id'))

    idx = site_pre_T['donor_id'].unique()
    out = pd.DataFrame(index=idx)

    # --- Recent activity metrics (3m window) ---
    if not site_3m.empty:
        # Days with any activity
        activity_by_day = (
            site_3m
            .assign(any_activity=1)
            .groupby(['donor_id', 'activity_date'])['any_activity']
            .max()
            .reset_index()
        )
        days_with_any = activity_by_day.groupby('donor_id')['activity_date'].nunique()
        
        # Session counts
        sessions_by_donor = site_3m.groupby('donor_id').size()
        
        out['days_with_any_site_activity_3m'] = days_with_any
        out['avg_sessions_per_active_day_3m'] = (
            sessions_by_donor / (days_with_any + 1e-6)
        )
        
        # Session duration (if available)
        if 'session_duration_min' in site_3m.columns:
            out['avg_session_duration_min_3m'] = (
                site_3m.groupby('donor_id')['session_duration_min'].mean()
            )
        
        # Checkout intent: cart visits per session
        if 'cart_visits_day' in site_3m.columns:
            out['checkout_intent_min_per_session_3m'] = (
                site_3m.groupby('donor_id')['cart_visits_day'].sum()
                / (sessions_by_donor + 1e-6)
            )
        
        # Campaign attribution
        if 'came_from_campaign' in site_3m.columns:
            out['campaign_session_share_3m'] = (
                site_3m.groupby('donor_id')['came_from_campaign'].mean()
            )
        
        # --- Page type mix ---
        # Ensure required columns exist with default 0
        for col in ['project_page_visits_day', 'teacher_page_visits_day', 'search_visits_day']:
            if col not in site_3m.columns:
                site_3m[col] = 0
        
        page_counts = site_3m.groupby('donor_id').agg({
            'project_page_visits_day': 'sum',
            'teacher_page_visits_day': 'sum',
            'search_visits_day'      : 'sum'
        })
        total_page_visits = page_counts.sum(axis=1) + 1e-6
        
        out['share_project_page_session_pct_3m'] = (
            page_counts['project_page_visits_day'] / total_page_visits
        )
        out['share_teacher_page_session_pct_3m'] = (
            page_counts['teacher_page_visits_day'] / total_page_visits
        )
        out['share_search_page_session_pct_3m'] = (
            page_counts['search_visits_day'] / total_page_visits
        )
        
        # --- Device profile ---
        # Mobile-first, desktop-only, or mixed?
        if 'device_type' in site_3m.columns:
            # Normalize device_type to lowercase to match ['mobile', 'desktop', 'tablet']
            dev_df = site_3m.copy()
            dev_df['device_type'] = dev_df['device_type'].astype(str).str.lower()
            
            device_counts = (
                dev_df.groupby(['donor_id', 'device_type'])
                .size()
                .unstack(fill_value=0)
            )
            total_device = device_counts.sum(axis=1) + 1e-6
            
            for dev in ['mobile', 'desktop', 'tablet']:
                colname = f'device_share_{dev}_3m'
                if dev in device_counts.columns:
                    out[colname] = device_counts[dev] / total_device
                else:
                    out[colname] = 0.0

    # --- Cart recency (using full history, relative to T) ---
    if 'cart_visits_day' in site_pre_T.columns:
        has_cart = site_pre_T[site_pre_T['cart_visits_day'] > 0]
        if not has_cart.empty:
            last_cart_date = has_cart.groupby('donor_id')['activity_date'].max()
            out['days_since_last_cart_visit'] = (
                (T - last_cart_date).dt.days
            )
    
    return out

### 2.10 Share Events

In [12]:
def _share_features(share_pre_T, share_12m, T):
    """
    Build social sharing behavior features.
    
    Features created:
    - share_events_lifetime, share_events_12m: Volume
    - share_active_months_12m: Frequency
    - share_gap_mean/cv_days: Sharing rhythm
    - share_month_coverage_ratio: Consistency over tenure
    
    Parameters
    ----------
    share_pre_T : pd.DataFrame
        All share events before T
    share_12m : pd.DataFrame
        Share events in last 12 months
    T : pd.Timestamp
        Reference time
        
    Returns
    -------
    features : pd.DataFrame
        Feature matrix indexed by donor_id
    """
    if share_pre_T.empty:
        return pd.DataFrame(index=pd.Index([], name='donor_id'))
    
    out_index = share_pre_T['donor_id'].unique()
    out = pd.DataFrame(index=out_index)
    
    # --- Volume metrics ---
    out['share_events_lifetime'] = (
        share_pre_T.groupby('donor_id')['share_event_count'].sum()
    )
    out['share_events_12m'] = (
        share_12m.groupby('donor_id')['share_event_count'].sum()
        if not share_12m.empty else 0
    )
    
    # --- Active months (consistency) ---
    if not share_12m.empty:
        active_months_12m = (
            share_12m[share_12m['share_event_count'] > 0]
            .groupby('donor_id')['share_month_start']
            .nunique()
        )
        out['share_active_months_12m'] = active_months_12m
    
    # --- Sharing rhythm: gaps between share months ---
    def gap_stats(s):
        if s.shape[0] < 2:
            return pd.Series({
                'share_gap_mean_days': np.nan,
                'share_gap_cv_days': np.nan
            })
        # Sort by time, then diff
        gaps = s.sort_values().diff().dropna().dt.days
        mean = gaps.mean()
        cv = gaps.std() / (mean + 1e-6)
        return pd.Series({
            'share_gap_mean_days': mean,
            'share_gap_cv_days': cv
        })

    gap_df = (
        share_pre_T.groupby('donor_id')['share_month_start']
        .apply(gap_stats)
        .unstack()          # columns: share_gap_mean_days, share_gap_cv_days
    )

    out = out.join(gap_df, how='left')

    # --- Coverage ratio ---
    first_share = share_pre_T.groupby('donor_id')['share_month_start'].min()
    last_share = share_pre_T.groupby('donor_id')['share_month_start'].max()
    
    # Difference of two Periods is a DateOffset (e.g., <MonthEnd>); use .n to get month count
    tenure_offsets = last_share.dt.to_period('M') - first_share.dt.to_period('M')
    tenure_months = tenure_offsets.apply(lambda x: x.n if pd.notnull(x) else 0)
    tenure_months = tenure_months.clip(lower=1)  # at least 1 month of tenure
    
    out['share_month_coverage_ratio'] = (
        out.get('share_active_months_12m', 0) / (tenure_months + 1e-6)
    )

    return out

### 2.11 Same Schoool & Teacher Available Flags

In [13]:
def _future_opportunity_features(dpr_pre_T, df_project_dates, T, H):
    """
    Future opportunity features based on the donor's TOP (most-funded) school and teacher,
    not just their first one.

    For each donor:
      - Find the school where they have given the most (by payment_amount).
      - Find the teacher where they have given the most (by payment_amount).
      - Flag whether that school/teacher has ANY project active in [T, T+H].

    This approximates "did the donor's favorite school/teacher have opportunities
    to receive more gifts during the horizon".
    """
    # If we don't have project data or donations, return zeros
    if df_project_dates is None or df_project_dates.empty or dpr_pre_T.empty:
        idx = dpr_pre_T['donor_id'].unique()
        return pd.DataFrame({
            'school_still_available_during_range': 0,
            'teacher_still_available_during_range': 0
        }, index=pd.Index(idx, name='donor_id'))

    # Work on copies
    proj = df_project_dates.copy()
    dpr = dpr_pre_T.copy()

    # Ensure project date columns are datetime
    for c in ['project_last_posted_date', 'project_funded_date', 'project_expiration_date']:
        if c in proj.columns:
            proj[c] = pd.to_datetime(proj[c], errors='coerce')

    # Project end date = earlier of funded or expired
    proj['end_date'] = proj[['project_funded_date', 'project_expiration_date']].min(axis=1)

    # Normalize H
    if isinstance(H, int):
        H = pd.Timedelta(days=H)
    elif H is None:
        H = pd.Timedelta(days=365)

    win_start = T
    win_end = T + H

    # ----------------------------------------------------------------------
    # 1) Identify TOP (most-funded) school and teacher per donor
    # ----------------------------------------------------------------------
    donor_index = dpr['donor_id'].unique()
    out = pd.DataFrame(index=pd.Index(donor_index, name='donor_id'))

    # Top school by total payment_amount
    if {'school_id', 'payment_amount'}.issubset(dpr.columns):
        school_agg = (
            dpr.groupby(['donor_id', 'school_id'])['payment_amount']
            .sum()
            .reset_index()
        )
        top_school = (
            school_agg
            .sort_values(['donor_id', 'payment_amount'], ascending=[True, False])
            .groupby('donor_id')
            .first()
            .reset_index()[['donor_id', 'school_id']]
        )
    else:
        top_school = pd.DataFrame(columns=['donor_id', 'school_id'])

    # Top teacher by total payment_amount
    if {'teacher_id', 'payment_amount'}.issubset(dpr.columns):
        teacher_agg = (
            dpr.groupby(['donor_id', 'teacher_id'])['payment_amount']
            .sum()
            .reset_index()
        )
        top_teacher = (
            teacher_agg
            .sort_values(['donor_id', 'payment_amount'], ascending=[True, False])
            .groupby('donor_id')
            .first()
            .reset_index()[['donor_id', 'teacher_id']]
        )
    else:
        top_teacher = pd.DataFrame(columns=['donor_id', 'teacher_id'])

    # If a donor has only one gift, top == first; so behavior matches what you described.

    # ----------------------------------------------------------------------
    # 2) Precompute which schools/teachers have projects active in horizon
    # ----------------------------------------------------------------------
    proj['active_in_horizon'] = (
        (proj['project_last_posted_date'] <= win_end) &
        (proj['end_date'] >= win_start)
    )

    # Guard against missing columns, though in your data they should exist
    if 'school_id' in proj.columns:
        school_active = (
            proj.groupby('school_id')['active_in_horizon']
            .any()
            .astype(int)
        )
    else:
        school_active = pd.Series(dtype=int)

    if 'teacher_id' in proj.columns:
        teacher_active = (
            proj.groupby('teacher_id')['active_in_horizon']
            .any()
            .astype(int)
        )
    else:
        teacher_active = pd.Series(dtype=int)

    # ----------------------------------------------------------------------
    # 3) Map top school/teacher to these activity flags
    # ----------------------------------------------------------------------
    # School: donor -> top school_id -> activity flag
    if not top_school.empty:
        school_flag = (
            top_school
            .set_index('donor_id')['school_id']
            .map(school_active)
            .reindex(donor_index)
            .fillna(0)
            .astype(int)
        )
    else:
        school_flag = pd.Series(0, index=donor_index)

    # Teacher: donor -> top teacher_id -> activity flag
    if not top_teacher.empty:
        teacher_flag = (
            top_teacher
            .set_index('donor_id')['teacher_id']
            .map(teacher_active)
            .reindex(donor_index)
            .fillna(0)
            .astype(int)
        )
    else:
        teacher_flag = pd.Series(0, index=donor_index)

    out['school_still_available_during_range'] = school_flag
    out['teacher_still_available_during_range'] = teacher_flag

    return out

### 2.11 Project Outcomes & Matching

In [14]:
def _project_outcome_features(dpr_pre_T):
    """
    Project outcome and matching features at the donor level.
    Features:
    - pct_projects_fully_funded: share of DONATIONS (not projects) that went to fully funded projects
    - mean_project_total_cost: average cost of projects supported (donation-weighted)
    - median_donor_to_project_distance_mi: median distance to projects
    - pct_gifts_within_15mi: share of gifts to local projects (<15 miles)
    - is_local_donor: binary flag for primarily local giving (>50% within 15mi)
    
    Note: Without project_id, we calculate at donation level rather than project level.
    """
    if dpr_pre_T.empty:
        return pd.DataFrame(index=pd.Index([], name='donor_id'))
    
    dpr = dpr_pre_T.copy()
    donor_idx = pd.Index(dpr['donor_id'].unique(), name='donor_id')
    out = pd.DataFrame(index=donor_idx)
    
    # --- Project funding outcomes (donation-level) ---
    if 'project_got_fully_funded' in dpr.columns:
        # This is now "% of donations that went to fully funded projects"
        # (not % of unique projects that got funded)
        out['pct_projects_fully_funded'] = (
            dpr.groupby('donor_id')['project_got_fully_funded']
            .mean()
            .reindex(donor_idx)
        )
    else:
        out['pct_projects_fully_funded'] = np.nan
    
    # --- Average project cost (donation-weighted) ---
    if 'project_total_cost' in dpr.columns:
        # Mean project cost across all donations (weighted by # of donations per project)
        out['mean_project_total_cost'] = (
            dpr.groupby('donor_id')['project_total_cost']
            .mean()
            .reindex(donor_idx)
        )
    else:
        out['mean_project_total_cost'] = np.nan

    # Median project cost
    if not dpr_pre_T.empty and 'project_total_cost' in dpr_pre_T.columns:
        median_cost = dpr_pre_T.groupby('donor_id')['project_total_cost'].median()
        out['median_project_total_cost'] = median_cost
    else:
        out['median_project_total_cost'] = np.nan
    
    # Match multiplier features
    if 'match_xyi_multiplier' in dpr.columns:
        # Mean match multiplier (donor-weighted)
        out['mean_match_multiplier'] = (
            dpr.groupby('donor_id')['match_xyi_multiplier']
            .mean()
            .reindex(donor_idx)
        )
    
        # Percentage of gifts with any match (match_xyi_multiplier > 1)
        match_flag = dpr['match_xyi_multiplier'] > 1
        out['pct_gifts_with_match'] = (
            match_flag.groupby(dpr['donor_id'])
            .mean()  # fraction of donations with match
            .reindex(donor_idx)
        )
    else:
        out['mean_match_multiplier'] = np.nan
        out['pct_gifts_with_match'] = np.nan    

    # --- Geographic distance features ---
    if 'distance_mi' in dpr.columns:
        out['median_donor_to_project_distance_mi'] = (
            dpr.groupby('donor_id')['distance_mi'].median().reindex(donor_idx)
        )
        
        # Local giving patterns (within 15 miles)
        dpr_with_distance = dpr[dpr['distance_mi'].notna()].copy()
        if not dpr_with_distance.empty:
            dpr_with_distance['within_15mi'] = (dpr_with_distance['distance_mi'] <= 15).astype(int)
            
            out['pct_gifts_within_15mi'] = (
                dpr_with_distance.groupby('donor_id')['within_15mi'].mean().reindex(donor_idx)
            )
            
            # Binary flag: is this a "local donor" (>50% of gifts within 15mi)
            out['is_local_donor'] = (out['pct_gifts_within_15mi'] > 0.5).astype(int)
        else:
            out['pct_gifts_within_15mi'] = np.nan
            out['is_local_donor'] = np.nan
    else:
        out['median_donor_to_project_distance_mi'] = np.nan
        out['pct_gifts_within_15mi'] = np.nan
        out['is_local_donor'] = np.nan
    
    return out

### 2.12 Labels (Optional)

In [15]:
def _build_labels(df_dpr, df_monthly, df_share, T, H):
    """
    Build label/target variables for future horizon H.
    
    This is a basic implementation covering repeat giving labels.
    You can extend this for:
    - Monthly program labels (became_monthly, churned_monthly)
    - Share labels (shared_in_H)
    - Upgrade/downgrade labels
    - High-value donor labels
    
    Parameters
    ----------
    df_dpr : pd.DataFrame
        Complete donation records (not pre-filtered)
    df_monthly : pd.DataFrame
        Monthly subscription records
    df_share : pd.DataFrame
        Share events
    T : pd.Timestamp
        Start of label window
    H : pd.Timedelta or int
        Length of label window (e.g., 365 days for 12-month prediction)
        
    Returns
    -------
    labels : pd.DataFrame
        Label matrix indexed by donor_id
    """
    T = pd.to_datetime(T)
    if isinstance(H, int):
        H = pd.Timedelta(days=H)
    
    start = T
    end = T + H
    
    # --- Repeat giving labels ---
    df_dpr = df_dpr.copy()
    df_dpr['payment_date'] = pd.to_datetime(df_dpr['payment_date'])
    
    # Filter to label window
    dpr_label = df_dpr[
        (df_dpr['payment_date'] >= start) &
        (df_dpr['payment_date'] < end)
    ]
    
    by_label = dpr_label.groupby('donor_id')['payment_amount']
    
    # Basic repeat giving labels
    gave_any_in_H = by_label.size().gt(0).astype(int)
    gift_count_in_H = by_label.size()
    gift_amount_in_H = by_label.sum()
    median_gift_in_H = by_label.median()
    
    labels = pd.DataFrame(index=gave_any_in_H.index)
    labels['gave_any_in_H'] = gave_any_in_H
    labels['gift_count_in_H'] = gift_count_in_H
    labels['gift_amount_in_H'] = gift_amount_in_H
    labels['median_gift_amount_in_H'] = median_gift_in_H
    
    # Could add more labels here:
    # - became_monthly_in_H (from df_monthly)
    # - churned_monthly_in_H
    # - shared_in_H (from df_share)
    # - upgrade_in_H (median_gift_in_H > median_gift_12m)
    
    return labels

## 3. Main Build Features Function

This orchestrates all the feature groups.

In [16]:
def build_features(
    df_dpr,
    df_email,
    df_site,
    df_monthly,
    df_share,
    df_zip_acs,
    df_project_dates,
    eligibility_months,
    T,
    H=None
):
    """
    Build complete donor-level feature matrix as of time T.
    
    This is the main entry point for feature engineering. It:
    1. Normalizes all timestamps
    2. Filters data to events before T
    3. Creates windowed subsets (3m, 12m, 36m)
    4. Calls helper functions for each feature group
    5. Optionally builds labels for horizon H
    
    IMPORTANT: This does NOT do final imputation or encoding.
    You should call finalize_features() after this to handle:
    - Missing value imputation
    - Categorical encoding
    - Feature scaling (if desired)
    
    Parameters
    ----------
    df_dpr : pd.DataFrame
        Donor Project Records with columns:
        - donor_id, payment_date, payment_amount, donation_n
        - donor_zip, is_teacher, is_teacher_referred
        - teacher_id, school_id, project_id, project_category, etc.
    df_email : pd.DataFrame
        Email Events 12mo with columns:
        - donor_id, email_sent_month
        - email_sent_count, email_open_count, email_click_count
    df_site : pd.DataFrame
        Site Events with columns:
        - donor_id, activity_date
        - device_type, came_from_campaign
        - project_page_visits_day, teacher_page_visits_day, etc.
    df_monthly : pd.DataFrame
        Monthly DonationLevel with columns:
        - donor_id, monthly_subscription_joined_date
        - monthly_subscription_retired_date
        - monthly_subscription_payment_amount, charge_date
    df_share : pd.DataFrame
        Share Events with columns:
        - donor_id, share_sent_month, share_event_count
    df_zip_acs : pd.DataFrame
        ZIP-level ACS demographics with column:
        - ZIP5 (index), pct_households_with_children, unemployment_rate, etc.
    T : pd.Timestamp or str
        Reference time (as-of date) for feature computation
        All features use only data strictly before this timestamp
    H : pd.Timedelta, int, or None
        Label horizon (e.g., pd.Timedelta(days=365) or 365)
        If provided, labels will be computed for window [T, T+H)
        
    Returns
    -------
    features : pd.DataFrame
        Donor-level feature matrix indexed by donor_id
        Contains ~150 columns across 11 feature groups
        May contain NaN values that should be imputed
        
    Example
    -------
    >>> T = pd.Timestamp('2024-01-01')
    >>> H = pd.Timedelta(days=365)  # 12-month prediction
    >>> features = build_features(
    ...     df_dpr, df_email, df_site, df_monthly, df_share, df_zip_acs,
    ...     T=T, H=H
    ... )
    >>> print(features.shape)
    (100000, 152)  # 100k donors, 152 features
    """
    # =====================================================================
    # SETUP: Convert dates and normalize T
    # =====================================================================
    
    T = pd.to_datetime(T)
    
    # Convert all date columns upfront (before eligibility filtering)
    df_dpr = df_dpr.copy()
    df_dpr['payment_date'] = pd.to_datetime(df_dpr['payment_date'])
    
    df_monthly = df_monthly.copy()
    df_monthly['monthly_subscription_joined_date'] = pd.to_datetime(
        df_monthly['monthly_subscription_joined_date']
    )
    df_monthly['monthly_subscription_retired_date'] = pd.to_datetime(
        df_monthly['monthly_subscription_retired_date']
    )
    if 'charge_date' in df_monthly.columns:
        df_monthly['charge_date'] = pd.to_datetime(df_monthly['charge_date'])
    
    # =====================================================================
    # ELIGIBILITY FILTERING (if requested)
    # =====================================================================
    
    if eligibility_months is not None:
        eligibility_start = T - pd.DateOffset(months=eligibility_months)
        
        # Donors with project donations in eligibility window
        active_project_donors = df_dpr[
            (df_dpr['payment_date'] >= eligibility_start) &
            (df_dpr['payment_date'] < T)
        ]['donor_id'].unique()
        
        # Donors active in monthly program during eligibility window
        active_monthly_donors = df_monthly[
            (df_monthly['monthly_subscription_joined_date'] < T) &
            (df_monthly['monthly_subscription_retired_date'].isna() | 
             (df_monthly['monthly_subscription_retired_date'] >= eligibility_start))
        ]['donor_id'].unique()
        
        # Union of active donors
        base_donor_ids = pd.Index(
            pd.unique(
                pd.concat([
                    pd.Series(active_project_donors),
                    pd.Series(active_monthly_donors)
                ], ignore_index=True)
            ),
            name='donor_id'
        )        

    else:
        # Original behavior: all donors from both sources
        base_donor_ids = pd.Index(
            pd.unique(
                pd.concat([
                    df_dpr['donor_id'],
                    df_monthly['donor_id']
                ], ignore_index=True)
            ),
            name='donor_id'
        )
    
    # Create master feature dataframe
    features = pd.DataFrame(index=base_donor_ids)
    
    # Restrict all event tables to the base donor universe
    df_dpr = df_dpr[df_dpr['donor_id'].isin(base_donor_ids)].copy()
    df_email = df_email[df_email['donor_id'].isin(base_donor_ids)].copy()
    df_site = df_site[df_site['donor_id'].isin(base_donor_ids)].copy()
    df_monthly = df_monthly[df_monthly['donor_id'].isin(base_donor_ids)].copy()
    df_share = df_share[df_share['donor_id'].isin(base_donor_ids)].copy()

    # =====================================================================
    # NORMALIZE DATES & FILTER PRE-T
    # =====================================================================
    
    # DPR: main donation records
    df_dpr = df_dpr.copy()
    df_dpr['payment_date'] = pd.to_datetime(df_dpr['payment_date'])
    dpr_pre_T = df_dpr[df_dpr['payment_date'] < T].copy()


    # Precompute distances once using direct lat/lon → miles
    if {'donor_lat_long', 'school_lat_long'}.issubset(dpr_pre_T.columns):

        # Parse lat/long strings into numeric columns
        dpr_pre_T['donor_lat_long'] = dpr_pre_T['donor_lat_long'].astype(str)
        dpr_pre_T['school_lat_long'] = dpr_pre_T['school_lat_long'].astype(str)

        donor_lat_lon = dpr_pre_T['donor_lat_long'].str.split(',', expand=True)
        school_lat_lon = dpr_pre_T['school_lat_long'].str.split(',', expand=True)

        dpr_pre_T['donor_lat'] = donor_lat_lon[0].astype(float)
        dpr_pre_T['donor_lon'] = donor_lat_lon[1].astype(float)
        dpr_pre_T['school_lat'] = school_lat_lon[0].astype(float)
        dpr_pre_T['school_lon'] = school_lat_lon[1].astype(float)

        # Direct haversine distance from donor to project
        dpr_pre_T['distance_mi'] = haversine_miles(
            dpr_pre_T['donor_lat'],
            dpr_pre_T['donor_lon'],
            dpr_pre_T['school_lat'],
            dpr_pre_T['school_lon'],
        )
    else:
        dpr_pre_T['distance_mi'] = np.nan
    
    # Site events
    df_site = df_site.copy()
    df_site['activity_date'] = pd.to_datetime(df_site['activity_date'])
    site_pre_T = df_site[df_site['activity_date'] < T].copy()
    
    # Share events (monthly aggregates)
    df_share = df_share.copy()
    df_share['share_month_start'] = pd.to_datetime(
        df_share['share_sent_month']
    ).dt.to_period('M').dt.to_timestamp()
    share_pre_T = df_share[df_share['share_month_start'] < T].copy()
    
    # Email events (monthly aggregates)
    df_email = df_email.copy()
    df_email['email_month_start'] = pd.to_datetime(
        df_email['email_sent_month']
    ).dt.to_period('M').dt.to_timestamp()
    email_pre_T = df_email[df_email['email_month_start'] < T].copy()
    
    # Monthly subscriptions
    df_monthly = df_monthly.copy()
    df_monthly['monthly_subscription_joined_date'] = pd.to_datetime(
        df_monthly['monthly_subscription_joined_date']
    )
    df_monthly['monthly_subscription_retired_date'] = pd.to_datetime(
        df_monthly['monthly_subscription_retired_date']
    )
    if 'charge_date' in df_monthly.columns:
        df_monthly['charge_date'] = pd.to_datetime(df_monthly['charge_date'])
    
    md_pre_T = df_monthly[
        df_monthly['monthly_subscription_joined_date'] < T
    ].copy()
    
    # =====================================================================
    # WINDOW BOUNDARIES
    # =====================================================================
    # These define our lookback periods for windowed features
    
    W_short_start = T - pd.DateOffset(months=3)   # [T-3m, T)
    W_mid_start = T - pd.DateOffset(months=12)    # [T-12m, T)
    W_long_start = T - pd.DateOffset(months=36)   # [T-36m, T)
    
    # Create windowed DPR subsets for counts, amounts, velocities
    dpr_3m = dpr_pre_T[dpr_pre_T['payment_date'] >= W_short_start]
    dpr_12m = dpr_pre_T[dpr_pre_T['payment_date'] >= W_mid_start]
    dpr_36m = dpr_pre_T[dpr_pre_T['payment_date'] >= W_long_start]
    
    # Intermediate periods for velocity calculations
    dpr_3to12m = dpr_pre_T[
        (dpr_pre_T['payment_date'] >= W_mid_start) &
        (dpr_pre_T['payment_date'] < W_short_start)
    ]
    dpr_12to36m = dpr_pre_T[
        (dpr_pre_T['payment_date'] >= W_long_start) &
        (dpr_pre_T['payment_date'] < W_mid_start)
    ]
    
    # Email and site windows
    email_12m = email_pre_T[email_pre_T['email_month_start'] >= W_mid_start]
    email_3m = email_pre_T[email_pre_T['email_month_start'] >= W_short_start]
    site_3m = site_pre_T[site_pre_T['activity_date'] >= W_short_start]
    share_12m = share_pre_T[share_pre_T['share_month_start'] >= W_mid_start]
    
    # =====================================================================
    # BUILD FEATURE GROUPS
    # =====================================================================
    # Each join adds a group of related features
    # Using left joins to preserve all donor_ids
    
    # 1. Identity & ZIP / ACS demographics
    features = features.join(
        _identity_and_zip_features(dpr_pre_T, df_zip_acs, md_pre_T),
        how='left'
    )
    
    # 2. Lifetime giving behavior & tenure
    features = features.join(
        _lifetime_giving_features(dpr_pre_T),
        how='left'
    )
    
    # 3. Windowed giving & velocity trends
    features = features.join(
        _windowed_giving_features(
            dpr_pre_T, dpr_3m, dpr_12m, dpr_36m, dpr_3to12m, dpr_12to36m, T
        ),
        how='left'
    )
    
    # 4. Channel/payment type mix
    features = features.join(
        _channel_mix_features(dpr_pre_T, dpr_12m),
        how='left'
    )
    
    # 5. Monthly subscription program
    features = features.join(
        _monthly_features(md_pre_T, dpr_pre_T, dpr_12m, T),
        how='left'
    )
    
    # 6. Teacher/school/content preferences
    features = features.join(
        _teacher_school_features(dpr_pre_T),
        how='left'
    )
    
    # 7. Seasonality & rhythm patterns
    features = features.join(
        _seasonality_features(dpr_pre_T),
        how='left'
    )
    
    # 8. Email engagement
    features = features.join(
        _email_features(email_pre_T, email_3m, email_12m, T),
        how='left'
    )
    
    # 9. Site behavior
    features = features.join(
        _site_features(site_pre_T, site_3m, T),
        how='left'
    )
    
    # 10. Share events
    features = features.join(
        _share_features(share_pre_T, share_12m, T),
        how='left'
    )
    
    # 11. Project outcomes & matching
    features = features.join(
        _project_outcome_features(dpr_pre_T),
        how='left'
    )

    # 12) School/teacher availability
    if df_project_dates is not None and not df_project_dates.empty:
        horizon = H if H is not None else pd.Timedelta(days=365)
        features = features.join(
            _future_opportunity_features(dpr_pre_T, df_project_dates, T, horizon),
            how='left'
        )
    else:
        features['school_still_available_during_range'] = 0
        features['teacher_still_available_during_range'] = 0

    # =====================================================================
    # OPTIONAL: BUILD LABELS
    # =====================================================================
    # If H is provided, create labels for prediction horizon
    
    if H is not None:
        labels = _build_labels(df_dpr, df_monthly, df_share, T, H)
        features = features.join(labels, how='left')
    
    return features

## 4. Finalize Features Function

Handles imputation and encoding to make features model-ready.

In [17]:
def finalize_features(
    df_features,
    label_cols=None,
    numeric_impute_strategy="median",
    drop_original_categoricals=True,
):
    """
    Finalize raw features for modeling by handling missing values and encoding.
    
    This function:
    1. Separates labels from features (if specified)
    2. Identifies numeric vs categorical columns
    3. Imputes missing values:
       - Numeric: median or zero per column
       - Categorical: "UNK" string
    4. One-hot encodes categorical columns
    5. Returns clean X (features) and y (labels)
    
    Parameters
    ----------
    df_features : pd.DataFrame
        Raw feature matrix from build_features(), indexed by donor_id
        May contain NaN values and categorical columns
    label_cols : list of str or None
        Column names that should be treated as labels/targets
        (e.g., ['gave_any_in_H', 'gift_amount_in_H'])
        These will be returned as y and NOT encoded as features
    numeric_impute_strategy : {"median", "zero"}
        Strategy for imputing numeric NaNs:
        - "median": Fill with column median (robust to outliers)
        - "zero": Fill with 0.0 (appropriate for counts/flags)
    drop_original_categoricals : bool
        If True, drop original categorical columns after one-hot encoding
        (standard practice for modeling)
        If False, keep both original and encoded versions
        
    Returns
    -------
    X : pd.DataFrame
        Numeric feature matrix ready for modeling
        All NaNs imputed, all categoricals encoded
        Index aligned with df_features.index
    y : pd.DataFrame or None
        Label DataFrame if label_cols provided, else None
        Index aligned with df_features.index
        
    Example
    -------
    >>> features = build_features(..., H=365)
    >>> X, y = finalize_features(
    ...     features,
    ...     label_cols=['gave_any_in_H'],
    ...     numeric_impute_strategy='median'
    ... )
    >>> print(X.shape, y.shape)
    (100000, 148) (100000, 1)
    
    >>> # Now ready for modeling
    >>> from sklearn.ensemble import RandomForestClassifier
    >>> model = RandomForestClassifier()
    >>> model.fit(X, y['gave_any_in_H'])
    """
    df = df_features.copy()
    
    # =====================================================================
    # 0. SPLIT LABELS
    # =====================================================================
    
    if label_cols is None:
        label_cols = []
    
    # Filter to labels that actually exist in the dataframe
    label_cols = [c for c in label_cols if c in df.columns]
    
    if label_cols:
        y = df[label_cols].copy()
        df = df.drop(columns=label_cols)
    else:
        y = None
    
    # =====================================================================
    # 1. IDENTIFY COLUMN TYPES
    # =====================================================================
    
    # Numeric: includes bool (will be 0/1)
    numeric_cols = df.select_dtypes(
        include=[np.number, 'bool']
    ).columns.tolist()
    
    # Categorical: everything else (object, category, string)
    categorical_cols = df.columns.difference(numeric_cols).tolist()
    
    # =====================================================================
    # 2. IMPUTE NUMERIC COLUMNS
    # =====================================================================
    
    df_numeric = df[numeric_cols].copy()
    
    if numeric_impute_strategy == "median":
        # Per-column median, falling back to 0 if entire column is NaN
        medians = df_numeric.median(axis=0)
        medians = medians.fillna(0.0)
        df_numeric = df_numeric.fillna(medians)
    
    elif numeric_impute_strategy == "zero":
        # Simple zero fill
        df_numeric = df_numeric.fillna(0.0)
    
    else:
        raise ValueError(
            "numeric_impute_strategy must be 'median' or 'zero'"
        )
    
    # =====================================================================
    # 3. IMPUTE & ENCODE CATEGORICAL COLUMNS
    # =====================================================================
    
    if categorical_cols:
        df_cats = df[categorical_cols].copy()
        
        # Convert to string and fill missing with 'UNK'
        for col in categorical_cols:
            df_cats[col] = df_cats[col].astype("string")
            df_cats[col] = df_cats[col].fillna("UNK")
        
        # One-hot encode
        # Each unique value becomes a binary column
        df_cats_ohe = pd.get_dummies(
            df_cats,
            prefix=categorical_cols,
            dummy_na=False  # We already filled NaNs with 'UNK'
        )
        
        # Decide whether to keep original categorical columns
        if drop_original_categoricals:
            # Standard: only keep one-hot encoded versions
            X = pd.concat([df_numeric, df_cats_ohe], axis=1)
        else:
            # Keep both original and encoded
            X = pd.concat([df_numeric, df_cats, df_cats_ohe], axis=1)
    
    else:
        # No categorical columns
        X = df_numeric
    
    # =====================================================================
    # 4. ALIGN INDICES
    # =====================================================================
    
    X.index = df_features.index
    if y is not None:
        y.index = df_features.index
    
    return X, y

## 5. Example Usage

Here's how to use the pipeline with your data.

In [18]:
# Example workflow (uncomment and adapt to your data)

# 1. Load your data
df_dpr = pd.read_csv('/Users/matt.fritz/Desktop/DonorProjectRecords_251118.csv')
df_email = pd.read_csv('/Users/matt.fritz/Desktop/Email Events 36mo.csv')
df_site = pd.read_csv('/Users/matt.fritz/Desktop/Site Events FY25-26.csv')
df_monthly = pd.read_csv('/Users/matt.fritz/Desktop/Monthly Donation Data All Time.csv')
df_share = pd.read_csv('/Users/matt.fritz/Desktop/Share Events All Time.csv')
df_project_dates = pd.read_csv('/Users/matt.fritz/Desktop/Project Dates FY22-26.csv')
df_zip_acs = pd.read_csv('/Users/matt.fritz/Desktop/Merged_Zip_ACS_Demographics.csv')

# 2. Define reference time, horizon, and eligibility window
eligibility_months = 12             # Only include donors who gave in N months prior to T
T = pd.Timestamp('2025-11-13')      # As-of date
H = pd.Timedelta(days=365)          # 365-day prediction horizon

# 3. Build features
features = build_features(
    df_dpr=df_dpr,
    df_email=df_email,
    df_site=df_site,
    df_monthly=df_monthly,
    df_share=df_share,
    df_project_dates=df_project_dates,
    df_zip_acs=df_zip_acs,
    eligibility_months=eligibility_months,
    T=T,
    H=H
)

# 4. Finalize for modeling
# X, y = finalize_features(
#     features,
#     label_cols=['gave_any_in_H', 'gift_amount_in_H'],
#     numeric_impute_strategy='median'
# )

/var/folders/j3/jwjf6cwj7czdz1klxbhhjst80000gp/T/ipykernel_17941/1815413022.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dpr = pd.read_csv('/Users/matt.fritz/Desktop/DonorProjectRecords_251118.csv')
/var/folders/j3/jwjf6cwj7czdz1klxbhhjst80000gp/T/ipykernel_17941/1815413022.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_monthly = pd.read_csv('/Users/matt.fritz/Desktop/Monthly Donation Data All Time.csv')


Eligible donors (active in last 12 months): 32930
  From project donations: 23525
  From monthly program: 10412


/var/folders/j3/jwjf6cwj7czdz1klxbhhjst80000gp/T/ipykernel_17941/1923667419.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pct_amount_in_back_to_school = by_donor.apply(frac_amount_where(bts_mask))
/var/folders/j3/jwjf6cwj7czdz1klxbhhjst80000gp/T/ipykernel_17941/1923667419.py:51: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pct_amount_in_final_week_of_year = by_donor.apply(frac_amount_where(final_mask))
/var/f

In [19]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype

# --------------------------------------------------------------------
# 1. Split columns into numeric vs categorical
# --------------------------------------------------------------------
numeric_cols = features.select_dtypes(include=[np.number]).columns
categorical_cols = features.columns.difference(numeric_cols)

# --------------------------------------------------------------------
# 2. Numeric summary
# --------------------------------------------------------------------
def summarize_numeric(df, cols):
    rows = []
    n_rows = len(df)

    for col in cols:
        s = df[col]
        dtype = s.dtype

        if not is_numeric_dtype(s):
            continue  # skip anything weird that sneaks in

        null_count = s.isna().sum()
        pct_null = (null_count / n_rows * 100.0) if n_rows > 0 else np.nan

        rows.append({
            "feature": col,
            "dtype": str(dtype),
            "min": s.min(),
            "median": s.median(),
            "mean": s.mean(),
            "max": s.max(),
            "null_count": null_count,
            "percent_null": pct_null,
        })

    numeric_summary = pd.DataFrame(rows).sort_values("feature").reset_index(drop=True)

    # Format numeric stats and percents to xx.xx
    for c in ["min", "median", "mean", "max", "percent_null"]:
        numeric_summary[c] = (
            numeric_summary[c]
            .astype(float)
            .map(lambda x: f"{x:.2f}" if pd.notnull(x) else "")
        )

    return numeric_summary

# --------------------------------------------------------------------
# 3. Categorical summary
# --------------------------------------------------------------------
def summarize_categorical(df, cols):
    rows = []
    for col in cols:
        s = df[col]
        dtype = s.dtype

        n = len(s)
        null_mask = s.isna()
        n_null = null_mask.sum()
        pct_null = (n_null / n * 100.0) if n > 0 else np.nan

        s_nonnull = s[~null_mask]

        if s_nonnull.empty:
            top_value = None
            pct_top = np.nan
            n_unique = 0
        else:
            vc = s_nonnull.value_counts(dropna=True)
            top_value = vc.index[0]
            top_count = vc.iloc[0]
            n_unique = vc.size
            pct_top = (top_count / len(s_nonnull) * 100.0)

        rows.append({
            "feature": col,
            "dtype": str(dtype),
            "top_value": top_value,
            "percent_top_value": pct_top,
            "num_unique_values": n_unique,
            "null_count": n_null,
            "percent_null": pct_null,
        })

    cat_summary = pd.DataFrame(rows).sort_values("feature").reset_index(drop=True)

    # Format percents to xx.xx
    for c in ["percent_top_value", "percent_null"]:
        cat_summary[c] = (
            cat_summary[c]
            .astype(float)
            .map(lambda x: f"{x:.2f}" if pd.notnull(x) else "")
        )

    return cat_summary

# --------------------------------------------------------------------
# 4. Build summaries
# --------------------------------------------------------------------
numeric_summary = summarize_numeric(features, numeric_cols)
categorical_summary = summarize_categorical(features, categorical_cols)

# --------------------------------------------------------------------
# 5. Inspect
# --------------------------------------------------------------------
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(f"Numeric features: {len(numeric_summary)}")
    display(numeric_summary)

    print(f"Categorical features: {len(categorical_summary)}")
    display(categorical_summary)


Numeric features: 148


,feature,dtype,min,median,mean,max,null_count,percent_null
0,amount_velocity_0to12_vs_12to36,float64,0.01,0.84,3.61,14123.87,15567,47.27
1,amount_velocity_0to3_vs_3to12,float64,0.00,0.46,2.69,6481.50,17174,52.15
2,avg_optional_donation_rate_12m,float64,0.00,12.50,10.32,15.00,9707,29.48
3,avg_optional_donation_rate_lifetime,float64,0.00,13.50,11.37,18.00,9583,29.10
4,avg_sessions_per_active_day_3m,float64,1.00,1.00,1.00,1.00,12045,36.58
5,campaign_session_share_3m,float64,0.00,0.88,0.73,1.00,12045,36.58
6,checkout_intent_min_per_session_3m,float64,0.00,0.00,0.01,8.00,12045,36.58
7,count_velocity_0to12_vs_12to36,float64,0.01,0.90,2.05,158.00,15567,47.27
8,current_account_credit_balance,float64,-126.96,0.00,75.73,46339.44,9405,28.56
9,cv_gap_between_gifts_days,float64,0.00,1.79,2.07,36.49,10427,31.66


Categorical features: 2


,feature,dtype,top_value,percent_top_value,num_unique_values,null_count,percent_null
0,first_donation_date,datetime64[ns],2025-08-20,0.69,5654,9405,28.56
1,last_donation_date,datetime64[ns],2025-10-31,5.41,361,9405,28.56


In [20]:
# Convert date first
df_dpr['payment_date'] = pd.to_datetime(df_dpr['payment_date'])

# Now create dpr_pre_T
dpr_pre_T = df_dpr[df_dpr['payment_date'] < T].copy()

print("\nDistance calculation coverage:")
print(f"Total donors: {len(features)}")
print(f"Records with donor_lat_long: {dpr_pre_T['donor_lat_long'].notna().sum()}")
print(f"Records with school_lat_long: {dpr_pre_T['school_lat_long'].notna().sum()}")
print(f"Records with both: {(dpr_pre_T['donor_lat_long'].notna() & dpr_pre_T['school_lat_long'].notna()).sum()}")
print(f"Donors with any distance data: {dpr_pre_T.groupby('donor_id')['donor_lat_long'].apply(lambda x: x.notna().any()).sum()}")


Distance calculation coverage:
Total donors: 32930
Records with donor_lat_long: 1907266
Records with school_lat_long: 1989672
Records with both: 1746473
Donors with any distance data: 19744


## 6. Feature Inventory

Quick reference of all features by category:

### Identity & Demographics (17 features)
- donor_zip5, is_teacher, is_teacher_referred
- is_marketing_subscribed, is_major_gift_donor
- ever_used_account_credit, current_account_credit_balance
- zip_* (9 ACS features)

### Lifetime Giving (14 features)
- first_donation_date, last_donation_date
- tenure_days, tenure_years, tenure_bucket
- lifetime_gift_count, lifetime_amount
- lifetime_median/max/cv_gift_amount
- mean/cv_gap_between_gifts_days
- max_donation_sequence_number
- pct_early_gifts_in_lifetime

### Windowed Giving (12 features)
- gift_count/amount/median_amount for 3m, 12m, 36m
- days_since_last/second_to_last_gift
- amount_velocity_0to3_vs_3to12
- amount/count_velocity_0to12_vs_12to36

### Channel Mix (26 features)
- pct_amount/count_daf (lifetime, 12m)
- pct_amount/count_green (lifetime, 12m)
- pct_gifts/amount_gift_card (lifetime, 12m)
- pct_amount/count_big_event (lifetime, 12m)
- avg_optional_donation_rate (lifetime, 12m)
- pct_gifts_anonymous (lifetime, 12m)
- pct_amount_classroom_essentials (lifetime, 12m)

### Monthly Program (10 features)
- is_monthly_donor_current
- monthly_lifetime_amount, monthly_amount_12m
- monthly_median_gift_amount
- pct_amount_monthly (lifetime, 12m)
- months_on_program, months_since_last_monthly_charge
- monthly_longest_streak_months
- monthly_joined_before_first_project_gift

### Teacher/School Preferences (17 features)
- entropy_* (5: teacher, school, zip, category, grade)
- num_unique_* (5)
- pct_amount_to_top_* (4)
- pct_gifts_first/last_project
- mean_teacher_lifetime_projects_fully_funded
- mean_teacher_lifetime_donations

### Seasonality (10 features)
- pct_amount_in_back_to_school
- pct_amount_in_final_week_of_year
- pct_amount_on_weekends
- entropy_gift_month
- pct_amount_in_top_month/quarter
- first_donation_month/quarter
- first_donation_dow_sin/cos

### Email Engagement (11 features)
- emails_sent/opened/clicked (3m, 12m)
- email_open/click_rate (3m, 12m)
- email_open_rate_velocity_3m_vs_12m
- days_since_last_email_sent

### Site Behavior (11 features)
- days_with_any_site_activity_3m
- avg_sessions_per_active_day_3m
- avg_session_duration_min_3m
- checkout_intent_min_per_session_3m
- days_since_last_cart_visit
- campaign_session_share_3m
- share_*_page_session_pct_3m (3)
- device_share_*_3m (3)

### Share Events (5 features)
- share_events_lifetime/12m
- share_active_months_12m
- share_gap_mean/cv_days
- share_month_coverage_ratio

### Project Outcomes (8 features)
- pct_projects_fully_funded
- mean/median_project_total_cost
- mean_match_multiplier
- pct_gifts_with_match
- median_donor_to_project_distance_mi
- pct_gifts_within_15mi
- is_local_donor

### Labels (4 features, if H provided)
- gave_any_in_H
- gift_count_in_H
- gift_amount_in_H
- median_gift_amount_in_H

**Total: ~145+ base features** (before one-hot encoding categoricals)

## 7. Notes & Best Practices

### Time Scope Summary

- **STATIC**: Does not depend on T
  - ZIP/ACS features
  - Some "once ever" flags (is_major_gift_donor, etc.)
  - First donation seasonality

- **AS_OF_T**: Cumulative through time T
  - Lifetime giving metrics
  - Distance/locality
  - Channel mix (lifetime)
  - Teacher/school concentration
  - Overall seasonality patterns

- **WINDOWED**: Specific lookback relative to T
  - 3m/12m/36m counts & amounts
  - Velocity metrics
  - Email engagement (3m/12m)
  - Site behavior (3m)
  - Share activity (12m)

- **LABEL**: Future horizon [T, T+H)
  - Repeat giving targets
  - Monthly program changes
  - Upgrade/downgrade indicators

### Key Design Decisions

1. **No duplicate concepts**: Single distance metric, single velocity pattern
2. **Consistent windows**: 3m (short), 12m (mid), 36m (long)
3. **Defensive epsilon**: Small values (1e-6) prevent division by zero
4. **Separation of concerns**: Feature engineering ≠ imputation
5. **Time-relative**: All features parameterized by T

### Recommended Workflow

1. **Build features** for each training example at different T values
2. **Finalize features** with consistent imputation strategy
3. **Feature selection** based on importance/correlation
4. **Model training** with cross-validation
5. **Production scoring** with latest T value

### Common Pitfalls to Avoid

- **Leakage**: Don't use features from [T, T+H) for predictions
- **Inconsistent windows**: Always use same T-relative boundaries
- **Missing joins**: Ensure all tables have donor_id
- **Scale mismatch**: Consider feature scaling for some models
- **Category explosion**: Monitor one-hot encoding dimension

### Performance Tips

- For large datasets, consider building features in chunks
- Cache intermediate results (windowed subsets)
- Use categorical dtypes to save memory
- Consider feature hashing for high-cardinality categoricals
- Profile your code to identify bottlenecks

### Extensions

This schema can be extended with:
- Interaction features (e.g., tenure × amount)
- Polynomial features for non-linear patterns
- Embedding features from text (project descriptions)
- Time-series features (rolling statistics)
- Graph features (donor networks)
- External data (economic indicators, events)